In [1]:
import pandas as pd
import numpy as np

In [132]:
# panel russia
# необходимо чтобы все колонки какие возможно в panel russia были заполнены 
df = pd.read_excel('C:/Users/ScMa9004/Scripts/Panel_russia_315.xlsx')

In [133]:
#обрезаем EOS и Z**
df = df[~df['type'].isin(['EOS', 'ZMM', 'ZMS', 'ZME', 'ZSM', 'ZPV', 'ZWS', 'ZKT', 'ZTK'])]

In [134]:
df = df.rename(columns = {'OP_FMCG': 'NEW_OP_FMCG', 
                          'OP_CIG': 'NEW_OP_CIG', 
                          'OP_BEER': 'NEW_OP_BEER'})

In [135]:
# IBD
ibd_f = pd.read_excel('C:/Users/ScMa9004/Documents/Panel/NEW PANEL/Cell_IBD.xlsx', sheet_name = 'FMCG')
ibd_c = pd.read_excel('C:/Users/ScMa9004/Documents/Panel/NEW PANEL/Cell_IBD.xlsx', sheet_name = 'CIG')
ibd_b = pd.read_excel('C:/Users/ScMa9004/Documents/Panel/NEW PANEL/Cell_IBD.xlsx', sheet_name = 'BEER')

In [136]:
df = df.fillna(0)

In [137]:
# create blank data frame for each index
fmcg = pd.DataFrame()
cig = pd.DataFrame()
beer = pd.DataFrame()

In [138]:
# create blank data frame for ops 
ops = pd.DataFrame()

In [139]:
# put into ops list of unique opses
ops['ops'] = df['ops'].drop_duplicates()

In [140]:
ops = ops.reset_index(drop = True)

In [141]:
# list of mcells
map_types = pd.read_excel('C:/Users/ScMa9004/Documents/Panel/NEW PANEL/FMCG_CIG_BEER.xlsx', sheet_name = 'index + mcellname')

In [142]:
# calculate step of recruitment for each mcell
UNI = map_types[['index', 'OP', 'UNIVERSE']]
UNI['step'] = UNI['UNIVERSE'] / UNI['OP']
UNI = UNI.rename(columns = {'index': 'MCELL_ID'})
UNI = UNI[['MCELL_ID', 'step']]
UNI.loc[UNI['step'] == 'inf', 'step'] = 9999

C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [143]:
#создаем столбец для приоретизации по типам тт
df['store_priority'] = np.nan
df.loc[df['type'].isin(['OMS', 'OSM', 'OPV', 'OLA', 'OMM', 'OKT', 'OOP', 'OPR', 'OME']), 'store_priority'] = 4
df.loc[df['type'].isin(['OWS']), 'store_priority'] = 2
df.loc[df['type'].isin(['OBS', 'OKS', 'DKC', 'DKP', 'DKM', 'FKM', 'FKC', 'FKF', 'FKT', 'OTK', 'OKP']), 'store_priority'] = 1
df.loc[df['store_priority'].isna(), 'store_priority'] = 3

FMCG prepare

In [144]:
fmcg = df.merge(ibd_f)

In [145]:
fmcg = pd.pivot_table(fmcg, values=['NEW_OP_FMCG', 'AP_FMCG', 'ops_bool', 'store_priority', 'ccode'],
                      index=['region_normalny', 'IBD_ID', 'CELL_ID_FMCG', 'MCELL_ID_FMCG', 'ops'],
                      aggfunc = {'NEW_OP_FMCG': np.sum,
                                 'AP_FMCG': np.sum,
                                 'ops_bool':  np.sum,
                                 'store_priority': np.sum, 
                                 'ccode': 'count'}).reset_index()

In [146]:
# разные типы тт имеют разный приоритет (store_priority) и в каждой ячейке могут быть типы с разным приоритетом
# под ячейкой подразумевается наименьший таксон разбиения - ops (микроселл, разделенный по опсам (см. сводную таблицу выше))
# показатель store_priority суммируется для каждого ops'a, а затем нормируется на кол-во физическизх тт в этом ops'e
fmcg['store_priority'] /= fmcg['ccode']
fmcg = fmcg.drop(columns='ccode')

In [147]:
# если в ячейке есть город, являющийся операционным центром - такая ячейка приоритезируется при прочих равных
fmcg.loc[fmcg['ops_bool'] > 0, 'ops_bool'] = 2
fmcg.loc[fmcg['ops_bool'] == 0, 'ops_bool'] = 1

In [148]:
# создается колонка AP_full_FMCG - здесь полная АП
# в колонку AP_FMCG перезаписывается АП без оверсэмпла
fmcg['AP_full_FMCG'] = fmcg['AP_FMCG']
fmcg.loc[fmcg['AP_FMCG'] > fmcg['NEW_OP_FMCG'], 'AP_FMCG'] = fmcg['NEW_OP_FMCG']

In [149]:
fmcg['recrut'] = fmcg['NEW_OP_FMCG'] - fmcg['AP_FMCG']

In [150]:
fmcg = fmcg.rename(columns = {'CELL_ID_FMCG': 'CELL_ID',
                              'MCELL_ID_FMCG': 'MCELL_ID',
                              'NEW_OP_FMCG': 'NEW_OP',
                              'AP_full_FMCG': 'AP',
                             'AP_FMCG': 'AP_NSE'})

CIG PREPARE

In [151]:
cig = df.merge(ibd_c)

In [152]:
cig = pd.pivot_table(cig, values=['NEW_OP_CIG', 'AP_CIG', 'ops_bool', 'store_priority', 'ccode'],
                     index = ['region_normalny', 'IBD_ID', 'CELL_ID_CIG', 'MCELL_ID_CIG', 'ops'],
                     aggfunc = {'NEW_OP_CIG': np.sum,
                                'AP_CIG': np.sum,
                                'ops_bool': np.sum, 
                               'store_priority': np.sum, 
                                 'ccode': 'count'}).reset_index()

In [153]:
cig['store_priority'] /= cig['ccode']
cig = cig.drop(columns='ccode')

In [154]:
cig.loc[cig['ops_bool'] > 0, 'ops_bool'] = 2
cig.loc[cig['ops_bool'] == 0, 'ops_bool'] = 1

In [155]:
cig['AP_full_CIG'] = cig['AP_CIG']

In [156]:
cig.loc[cig['AP_CIG'] > cig['NEW_OP_CIG'], 'AP_CIG'] = cig['NEW_OP_CIG']

In [157]:
cig['recrut'] = cig['NEW_OP_CIG'] - cig['AP_CIG']

In [158]:
cig = cig.rename(columns = {'CELL_ID_CIG': 'CELL_ID',
                              'MCELL_ID_CIG': 'MCELL_ID',
                              'NEW_OP_CIG': 'NEW_OP',
                              'OLD_OP_CIG': 'OLD_OP',
                              'AP_full_CIG': 'AP',
                           'AP_CIG': 'AP_NSE'})

BEER PREPARE

In [159]:
beer = df.merge(ibd_b)

In [160]:
beer = pd.pivot_table(beer, values=['NEW_OP_BEER', 'AP_BEER', 'ops_bool', 'store_priority', 'ccode'],
                     index=['region_normalny', 'IBD_ID', 'CELL_ID_BEER', 'MCELL_ID_BEER', 'ops'],
                     aggfunc = {'NEW_OP_BEER': np.sum,
                                'AP_BEER': np.sum,
                                'ops_bool':  np.sum, 
                               'store_priority': np.sum, 
                                 'ccode': 'count'}).reset_index()

In [161]:
beer['store_priority'] /= beer['ccode']
beer = beer.drop(columns='ccode')

In [162]:
beer.loc[beer['ops_bool'] > 0, 'ops_bool'] = 2
beer.loc[beer['ops_bool'] == 0, 'ops_bool'] = 1

In [163]:
beer['AP_full_BEER'] = beer['AP_BEER']

In [164]:
beer.loc[beer['AP_BEER'] > beer['NEW_OP_BEER'], 'AP_BEER'] = beer['NEW_OP_BEER']

In [165]:
beer['recrut'] = beer['NEW_OP_BEER'] - beer['AP_BEER']

In [166]:
beer = beer.rename(columns = {'CELL_ID_BEER': 'CELL_ID',
                              'MCELL_ID_BEER': 'MCELL_ID',
                              'NEW_OP_BEER': 'NEW_OP',
                              'AP_full_BEER': 'AP',
                             'AP_BEER': 'AP_NSE'})

RES FILE WITH NSE RATE

In [167]:
res = fmcg.append(cig, sort = False)

In [168]:
res = res.append(beer, sort = False)

In [169]:
res['drop'] = res['AP'] - res['AP_NSE']

In [170]:
res = res.reset_index(drop = True)

In [171]:
res['nse'] = np.nan

In [172]:
# a - наполненность мадрас селла
# x - наполненность микроселла
# b - наполненность опса (ячейки)
for i in res['CELL_ID'].unique():
        a = res['AP_NSE'][res['CELL_ID'] == i].sum() / res['NEW_OP'][res['CELL_ID'] == i].sum()
        if a < 0.1:
            a = '5'
        elif a < 0.25:
            a = '4'
        elif a < 0.5:
            a = '3'
        elif a < 0.75:
            a = '2'
        else:
            a = '1'
        for k in res['MCELL_ID'][res['CELL_ID'] == i].unique():
            x = res['AP_NSE'][res['MCELL_ID'] == k].sum() / res['NEW_OP'][res['MCELL_ID'] == k].sum()
            if x < 0.1:
                x = '5'
            elif x < 0.25:
                x = '4'
            elif x < 0.5:
                x = '3'
            elif x < 0.75:
                x = '2'
            else:
                x = '1'
            for j in res[res['MCELL_ID'] == k].index:
                b = res['AP_NSE'][j] / res['NEW_OP'][j]
                if b < 0.1:
                    b = '5'
                elif b < 0.25:
                    b = '4'
                elif b < 0.5:
                    b = '3'
                elif b < 0.75:
                    b = '2'
                else:
                    b = '1'
            res.loc[j, 'nse'] = a + '|' + x + '|' + b + '|' + str(res['store_priority'][j]) + '|' + str(res['ops_bool'][j])

C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """


In [173]:
# наименьший приоритет для оставшихся пустыми значений (деление на 0)
res.loc[res['nse'].isna(), 'nse'] = '1|1|1|1|1'

In [174]:
# расчет рейтинга (nse_rate) с применением весов
# наполненность мадрас селла - 1
# наполненность микроселла - 0.75
# наполненность опса - 0.5
# приоритетные типы тт - 0.25
# наличие опреационного центра в опсе - 0.25
nse = res['nse'].str.split('|', n = 5, expand = True)
nse = nse.astype('float')
    
nse['nse_rate'] = nse[0] + nse[1] * 0.75 + nse[2] * 0.5 + nse[3] * 0.25 + nse[4] * 0.25
res['nse_rate'] = nse['nse_rate']

In [175]:
# расчет рейтинга на сброс. логика зеркальная
for i in res['CELL_ID'].unique():
        a = res['AP'][res['CELL_ID'] == i].sum() / res['NEW_OP'][res['CELL_ID'] == i].sum()
        if a > 4:
            a = '5'
        elif a > 3:
            a = '4'
        elif a > 2:
            a = '3'
        elif a > 1:
            a = '2'
        else:
            a = '1'
        if (res['NEW_OP'][res['CELL_ID'] == i].sum() == 0) & (res['AP'][res['CELL_ID'] == i].sum() > 0):
            a = '5'
        for k in res['MCELL_ID'][res['CELL_ID'] == i].unique():
            x = res['AP'][res['MCELL_ID'] == k].sum() / res['NEW_OP'][res['MCELL_ID'] == k].sum()
            if x > 4:
                x = '5'
            elif x > 3:
                x = '4'
            elif x > 2:
                x = '3'
            elif x > 1:
                x = '2'
            else:
                x = '1'
            for j in res[res['MCELL_ID'] == k].index:
                b = res['AP'][j] / res['NEW_OP'][j]
                if b > 4:
                    b = '5'
                elif b > 3:
                    b = '4'
                elif b > 2:
                    b = '3'
                elif b > 1:
                    b = '2'
                else:
                    b = '1'
                if (res['NEW_OP'][j] == 0) & (res['AP'][j] > 0):
                    b = '5'
                res.loc[j, 'nse_drop'] = a + '|' + x + '|' + b + '|' + str(res['store_priority'][j]) + '|' + str(res['ops_bool'][j])

C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in true_divide
C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in true_divide
C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: divide by zero encountered in true_divide
C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:29: RuntimeWarning: divide by zero encountered in true_divide
C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in true_divide

In [176]:
nse_drop = res['nse_drop'].str.split('|', n = 5, expand = True)
nse_drop = nse_drop.astype('float')
    
nse_drop['nse_rate_drop'] = nse_drop[0] + nse_drop[1] * 0.75 + nse_drop[2] * 0.5 - nse_drop[3] * 0.25 - nse_drop[4] * 0.25
res['nse_rate_drop'] = nse_drop['nse_rate_drop']

In [177]:
# add step
res = res.merge(UNI)

In [178]:
# add column with indexes name
res.loc[res['MCELL_ID'] > 5999, 'NDX'] = 'BEER'
res.loc[res['MCELL_ID'] < 4000, 'NDX'] = 'FMCG'
res.loc[(4000 <= res['MCELL_ID']) & (res['MCELL_ID'] < 6000), 'NDX'] = 'CIG'

In [179]:
# decrease rate of cells with tiny recruitment step
res.loc[(res['step'] < 10) & (res['NDX'] == 'FMCG'), 'nse_rate'] -= 1.5
res.loc[(res['step'] < 5) & (res['NDX'].isin(['CIG', 'BEER'])), 'nse_rate'] -= 1.5

In [180]:
# drop 26th cluster (lukhovitsyn) ?
#for i in res['ops'][res['region_normalny'].isin(['MSO', 'Moscow'])].index:
#    try:
#        res['ops'][i].remove(26)
#    except:
#        pass

In [181]:
res = res.sort_values(by = 'nse_rate', ascending = False).reset_index(drop = True).reset_index()

In [182]:
# fill types column
for i in map_types.index:
    res.loc[res['MCELL_ID'] == map_types['index'][i], 'types'] = map_types['Type'][i]

In [183]:
res['market'] = np.nan

In [184]:
# определение рынка
for i in res.types.unique():
    if all(j in ['OTK', 'OKT', 'OPS'] for j in list(i.split(','))):
        res.loc[res['types'] == i, 'market'] = 'cig'
    elif all(j in ['DKC', 'DKM', 'DKP', 'ODC', 'OHC', 'OPR', 'OPH'] for j in list(i.split(','))):
        res.loc[res['types'] == i, 'market'] = 'drug'
    elif all(j in ['OWS', 'ODR', 'OKS', 'OPV', 'OMS', 'OME', 'OLA',\
                   'FKM', 'FKT', 'FKF', 'FKC', 'IMS', 'IML'] for j in list(i.split(','))):
        res.loc[res['types'] == i, 'market'] = 'food+cig'
    elif all(j in ['ODR', 'OKP', 'OOP'] for j in list(i.split(','))):
        res.loc[res['types'] == i, 'market'] = 'food'

In [193]:
res['reg_ndx'] = res['NDX'] + '-' + res['region_normalny']

In [185]:
#res.to_excel('trend_nabor.xlsx', index = False)

RASCHET

In [194]:
df = res

In [195]:
def trend(res, ops, n):
    
    # sort by nse_rate
    res = res.sort_values(by = 'nse_rate', ascending = False).reset_index(drop = True)
    
    #ротация по индексам внутри каждого региона не более 1.5% от AP
    index_rotation = pd.pivot_table(res,
                      index = ['reg_ndx'], 
                      values = ['AP'],
                      aggfunc = np.sum).reset_index()
    
    index_rotation['1.5%'] = round(index_rotation['AP'] * 0.015)
    index_rotation['1.5%_drop'] = index_rotation['1.5%']
    
    #ротация по каналам не более трех процентов
    chanel = pd.pivot_table(res,
              index = ['types'], 
              values = ['AP', 'AP_NSE', 'NEW_OP'],
              aggfunc = np.sum).reset_index()
    
    chanel['rot'] = (chanel['AP'] * 0.03) // 1
    chanel.loc[chanel['rot'] == 0, 'rot'] = 1
    
    # сюда будет добавляться нагрузка на опсы
    ops['nagruzka' + str(n)] = 0
    
    #ротация по регионам 1% 
    rot = pd.pivot_table(res,
              index = ['region_normalny'], 
              values = ['AP', 'AP_NSE', 'NEW_OP'],
              aggfunc = np.sum).reset_index()
    
    rot['rot'] = (rot['AP'] * 0.01) // 1
    # раньше ротация была кастомная для каждого региона. Может пригодиться
    #rot.loc[rot['region_normalny'].isin(['STH', 'St. Petersburg', 'Samara']), 'rot'] = (rot['AP'] * 0.007) // 1
    #rot.loc[rot['region_normalny'].isin(['Novosibirsk', 'Vladivostok']), 'rot'] = (rot['AP'] * 0.006) // 1
    #rot.loc[rot['region_normalny'].isin(['FST']), 'rot'] = (rot['AP'] * 0.009) // 1
    #rot.loc[rot['region_normalny'].isin(['Krasnodar']), 'rot'] = (rot['AP'] * 0.008) // 1
    
    #rot['nse_dobav_ochka'] = rot['AP_NSE'] / rot['NEW_OP']
    
    #res.loc[res['region_normalny'].isin([rot['region_normalny'][rot['nse_dobav_ochka'] < 0.965]]), 'nse_rate'] += 0.5
    
    #счетчик для IBD, когда он становится нулем - в этот IBD перестает приходить задание (и набор и сброс)
    res['ibd_10'] = 10
    
    res['target'] = 0
    
    # 70 stores for recrut each step
    sto = 70
    
    while sto > 0:
        stop = 0
        for j in res.index:
            if sto > 0:
                if ((res['ibd_10'][res['IBD_ID'] == res['IBD_ID'][j]].unique().item() >= 1) &\
                (rot['rot'][rot['region_normalny'] == res['region_normalny'][j]] >= 1) &\
                (chanel['rot'][chanel['types'] == res['types'][j]].item() >= 1) &\
                (index_rotation['1.5%'][index_rotation['reg_ndx'] == res['reg_ndx'][j]].item() >= 1) &\
                (res['recrut'][j] > 0)).item():
                    res.loc[j, 'recrut'] -= 1
                    res.loc[j, 'target'] += 1
                    res.loc[res['IBD_ID'] == res['IBD_ID'][j], 'ibd_10'] -= 1
                    rot.loc[rot['region_normalny'] == res['region_normalny'][j], 'rot'] -= 1
                    chanel.loc[chanel['types'] == res['types'][j], 'rot'] -= 1
                    index_rotation['1.5%'][index_rotation['reg_ndx'] == res['reg_ndx'][j]] -= 1   
                    res.loc[j, 'AP_NSE'] += 1
                    res.loc[j, 'AP'] += 1
                    sto -= 1
                    ops.loc[ops['ops'] == res['ops'][j], 'nagruzka' + str(n)] += 1
                else:
                    stop += 1
                if stop == len(res):
                    sto = 0
                    break
            else:
                break
    # перерасчет коэффициента
    for i in res['CELL_ID'].unique():
        a = res['AP_NSE'][res['CELL_ID'] == i].sum() / res['NEW_OP'][res['CELL_ID'] == i].sum()
        if a < 0.1:
            a = '5'
        elif a < 0.25:
            a = '4'
        elif a < 0.5:
            a = '3'
        elif a < 0.75:
            a = '2'
        else:
            a = '1'
        for k in res['MCELL_ID'][res['CELL_ID'] == i].unique():
            x = res['AP_NSE'][res['MCELL_ID'] == k].sum() / res['NEW_OP'][res['MCELL_ID'] == k].sum()
            if x < 0.1:
                x = '5'
            elif x < 0.25:
                x = '4'
            elif x < 0.5:
                x = '3'
            elif x < 0.75:
                x = '2'
            else:
                x = '1'
            for j in res[res['MCELL_ID'] == k].index:
                b = res['AP_NSE'][j] / res['NEW_OP'][j]
                if b < 0.1:
                    b = '5'
                elif b < 0.25:
                    b = '4'
                elif b < 0.5:
                    b = '3'
                elif b < 0.75:
                    b = '2'
                else:
                    b = '1'
            res.loc[j, 'nse'] = a + '|' + x + '|' + b + '|' + str(res['store_priority'][j]) + '|' + str(res['ops_bool'][j])
    
    nse = res['nse'].str.split('|', n = 5, expand = True)
    nse = nse.astype('float')
    
    nse['nse_rate'] = nse[0] + nse[1] * 0.75 + nse[2] * 0.5 + nse[3] * 0.25 + nse[4] * 0.25
    
    res['nse_rate'] = nse['nse_rate']
    
    res.loc[(res['step'] < 10) & (res['NDX'] == 'FMCG'), 'nse_rate'] -= 1.5
    res.loc[(res['step'] < 5) & (res['NDX'].isin(['CIG', 'BEER'])), 'nse_rate'] -= 1.5
    
########################DROP#####################    
    
    chanel = pd.pivot_table(res,
              index = ['types'], 
              values = ['AP', 'AP_NSE', 'NEW_OP'],
              aggfunc = np.sum).reset_index()
    
    chanel['rot'] = (chanel['AP'] * 0.03) // 1
    chanel.loc[chanel['rot'] == 0, 'rot'] = 1

    ops['nagruzka_drop' + str(n)] = 0
    
    res['target_drop'] = 0
    
    for i in res['CELL_ID'].unique():
        a = res['AP'][res['CELL_ID'] == i].sum() / res['NEW_OP'][res['CELL_ID'] == i].sum()
        if a > 4:
            a = '5'
        elif a > 3:
            a = '4'
        elif a > 2:
            a = '3'
        elif a > 1:
            a = '2'
        else:
            a = '1'
        if (res['NEW_OP'][res['CELL_ID'] == i].sum() == 0) & (res['AP'][res['CELL_ID'] == i].sum() > 0):
            a = '5'
        for k in res['MCELL_ID'][res['CELL_ID'] == i].unique():
            x = res['AP'][res['MCELL_ID'] == k].sum() / res['NEW_OP'][res['MCELL_ID'] == k].sum()
            if x > 4:
                x = '5'
            elif x > 3:
                x = '4'
            elif x > 2:
                x = '3'
            elif x > 1:
                x = '2'
            else:
                x = '1'
            for j in res[res['MCELL_ID'] == k].index:
                b = res['AP'][j] / res['NEW_OP'][j]
                if b > 4:
                    b = '5'
                elif b > 3:
                    b = '4'
                elif b > 2:
                    b = '3'
                elif b > 1:
                    b = '2'
                else:
                    b = '1'
                if (res['NEW_OP'][j] == 0) & (res['AP'][j] > 0):
                    b = '5'
                res.loc[j, 'nse_drop'] = a + '|' + x + '|' + b + '|' + str(res['store_priority'][j]) + '|' + str(res['ops_bool'][j])
    
    nse_drop = res['nse_drop'].str.split('|', n = 5, expand = True)
    nse_drop = nse_drop.astype('float')
    
    nse_drop['nse_rate_drop'] = nse_drop[0] + nse_drop[1] * 0.75 + nse_drop[2] * 0.5 - nse_drop[3] * 0.25 - nse_drop[4] * 0.25
    
    res['nse_rate_drop'] = nse_drop['nse_rate_drop']
    
    res = res.sort_values(by = 'nse_rate_drop', ascending = False).reset_index(drop = True)
    
    rot = pd.pivot_table(res,
              index = ['region_normalny'], 
              values = ['AP', 'AP_NSE', 'NEW_OP'],
              aggfunc = np.sum).reset_index()
    
    rot['rot'] = (rot['AP'] * 0.01) // 1
    rot.loc[rot['rot'] == 0, 'rot'] = 1
    
    pis9t = 130
    
    while pis9t > 0:
        stop = 0
        for j in res.index:
            if pis9t > 0:
                if ((res['ibd_10'][res['IBD_ID'] == res['IBD_ID'][j]].unique().item() >= 1) &\
                (rot['rot'][rot['region_normalny'] == res['region_normalny'][j]] >= 1) &\
                (chanel['rot'][chanel['types'] == res['types'][j]].item() >= 1) &\
                (index_rotation['1.5%_drop'][index_rotation['reg_ndx'] == res['reg_ndx'][j]].item() >= 1) &\
                (res['drop'][j] > 0)).item():
                    res.loc[j, 'drop'] -= 1
                    res.loc[j, 'target_drop'] += 1
                    res.loc[res['IBD_ID'] == res['IBD_ID'][j], 'ibd_10'] -= 1
                    rot.loc[rot['region_normalny'] == res['region_normalny'][j], 'rot'] -= 1
                    index_rotation['1.5%_drop'][index_rotation['reg_ndx'] == res['reg_ndx'][j]] -= 1
                    chanel.loc[chanel['types'] == res['types'][j], 'rot'] -= 1
                    res.loc[j, 'AP'] -= 1
                    pis9t -= 1
                    ops.loc[ops['ops'] == res['ops'][j], 'nagruzka_drop' + str(n)] -= 1
                else:
                    stop += 1
                if stop == len(res):
                    pis9t = 0
                    print('vse')
                    break
            else:
                break
                
                
##########################непересекающиеся каналы############################
    
    
    #на каждый опс не более одной тт на рекрут и сброс
    normirovanyi_rabochy_den = pd.DataFrame()
    normirovanyi_rabochy_den['ops'] = res.ops.drop_duplicates()
    normirovanyi_rabochy_den['recrut'] = 1
    normirovanyi_rabochy_den['drop'] = 1
    
    #recrut
    res = res.sort_values(by = 'nse_rate', ascending = False).reset_index(drop = True)
    
    for i in res['region_normalny'].unique():
        for j in res['market'].dropna()[(res['region_normalny'] == i) & (res['target'] != 0)]:
            if j == 'cig':
                for pzdc in res['nse_rate'][(res['region_normalny'] == i) &
                                            (res['market'].isin(['drug', 'food'])) &
                                            (res['recrut'] > 0)].index:
                    if ((res['ibd_10'][res['IBD_ID'] == res['IBD_ID'][pzdc]].unique().item() >= 1) &\
                    (rot['rot'][rot['region_normalny'] == res['region_normalny'][pzdc]] >= 1) &\
                    (chanel['rot'][chanel['types'] == res['types'][pzdc]].item() >= 1) &\
                    (normirovanyi_rabochy_den['recrut'][normirovanyi_rabochy_den['ops'] == res['ops'][pzdc]].item() == 1) &\
                    (index_rotation['1.5%'][index_rotation['reg_ndx'] == res['reg_ndx'][pzdc]].item() >= 1)).item():
                        res.loc[pzdc, 'recrut'] -= 1
                        res.loc[pzdc, 'target'] += 1
                        res.loc[res['IBD_ID'] == res['IBD_ID'][pzdc], 'ibd_10'] -= 1
                        rot.loc[rot['region_normalny'] == res['region_normalny'][pzdc], 'rot'] -= 1
                        chanel.loc[chanel['types'] == res['types'][pzdc], 'rot'] -= 1
                        index_rotation['1.5%'][index_rotation['reg_ndx'] == res['reg_ndx'][pzdc]] -= 1 
                        normirovanyi_rabochy_den['recrut'][normirovanyi_rabochy_den['ops'] == res['ops'][pzdc]] -= 1
                        res.loc[pzdc, 'AP_NSE'] += 1
                        res.loc[pzdc, 'AP'] += 1
                        ops.loc[ops['ops'] == res['ops'][pzdc], 'nagruzka' + str(n)] += 1
                        continue
            elif j == 'drug':
                for pzdc in res['nse_rate'][(res['region_normalny'] == i) &
                                            (res['market'].isin(['cig', 'food'])) &
                                            (res['recrut'] > 0)].index:
                    if ((res['ibd_10'][res['IBD_ID'] == res['IBD_ID'][pzdc]].unique().item() >= 1) &\
                    (rot['rot'][rot['region_normalny'] == res['region_normalny'][pzdc]] >= 1) &\
                    (chanel['rot'][chanel['types'] == res['types'][pzdc]].item() >= 1) &\
                    (normirovanyi_rabochy_den['recrut'][normirovanyi_rabochy_den['ops'] == res['ops'][pzdc]].item() == 1) &\
                    (index_rotation['1.5%'][index_rotation['reg_ndx'] == res['reg_ndx'][pzdc]].item() >= 1)).item():
                        res.loc[pzdc, 'recrut'] -= 1
                        res.loc[pzdc, 'target'] += 1
                        res.loc[res['IBD_ID'] == res['IBD_ID'][pzdc], 'ibd_10'] -= 1
                        rot.loc[rot['region_normalny'] == res['region_normalny'][pzdc], 'rot'] -= 1
                        chanel.loc[chanel['types'] == res['types'][pzdc], 'rot'] -= 1
                        index_rotation['1.5%'][index_rotation['reg_ndx'] == res['reg_ndx'][pzdc]] -= 1 
                        normirovanyi_rabochy_den['recrut'][normirovanyi_rabochy_den['ops'] == res['ops'][pzdc]] -= 1
                        res.loc[pzdc, 'AP_NSE'] += 1
                        res.loc[pzdc, 'AP'] += 1
                        ops.loc[ops['ops'] == res['ops'][pzdc], 'nagruzka' + str(n)] += 1
                        continue
            elif j == 'food':
                for pzdc in res['nse_rate'][(res['region_normalny'] == i) &
                                            (res['market'].isin(['cig', 'drug'])) &
                                            (res['recrut'] > 0)].index:
                    if ((res['ibd_10'][res['IBD_ID'] == res['IBD_ID'][pzdc]].unique().item() >= 1) &\
                    (rot['rot'][rot['region_normalny'] == res['region_normalny'][pzdc]] >= 1) &\
                    (chanel['rot'][chanel['types'] == res['types'][pzdc]].item() >= 1) &\
                    (normirovanyi_rabochy_den['recrut'][normirovanyi_rabochy_den['ops'] == res['ops'][pzdc]].item() == 1) &\
                    (index_rotation['1.5%'][index_rotation['reg_ndx'] == res['reg_ndx'][pzdc]].item() >= 1)).item():
                        res.loc[pzdc, 'recrut'] -= 1
                        res.loc[pzdc, 'target'] += 1
                        res.loc[res['IBD_ID'] == res['IBD_ID'][pzdc], 'ibd_10'] -= 1
                        rot.loc[rot['region_normalny'] == res['region_normalny'][pzdc], 'rot'] -= 1
                        chanel.loc[chanel['types'] == res['types'][pzdc], 'rot'] -= 1
                        index_rotation['1.5%'][index_rotation['reg_ndx'] == res['reg_ndx'][pzdc]] -= 1 
                        normirovanyi_rabochy_den['recrut'][normirovanyi_rabochy_den['ops'] == res['ops'][pzdc]] -= 1
                        res.loc[pzdc, 'AP_NSE'] += 1
                        res.loc[pzdc, 'AP'] += 1
                        ops.loc[ops['ops'] == res['ops'][pzdc], 'nagruzka' + str(n)] += 1
                        continue
            elif j == 'food+cig':
                for pzdc in res['nse_rate'][(res['region_normalny'] == i) &
                                            (res['market'].isin(['drug'])) &
                                            (res['recrut'] > 0)].index:
                    if ((res['ibd_10'][res['IBD_ID'] == res['IBD_ID'][pzdc]].unique().item() >= 1) &\
                    (rot['rot'][rot['region_normalny'] == res['region_normalny'][pzdc]] >= 1) &\
                    (chanel['rot'][chanel['types'] == res['types'][pzdc]].item() >= 1) &\
                    (normirovanyi_rabochy_den['recrut'][normirovanyi_rabochy_den['ops'] == res['ops'][pzdc]].item() == 1) &\
                    (index_rotation['1.5%'][index_rotation['reg_ndx'] == res['reg_ndx'][pzdc]].item() >= 1)).item():
                        res.loc[pzdc, 'recrut'] -= 1
                        res.loc[pzdc, 'target'] += 1
                        res.loc[res['IBD_ID'] == res['IBD_ID'][pzdc], 'ibd_10'] -= 1
                        rot.loc[rot['region_normalny'] == res['region_normalny'][pzdc], 'rot'] -= 1
                        chanel.loc[chanel['types'] == res['types'][pzdc], 'rot'] -= 1
                        index_rotation['1.5%'][index_rotation['reg_ndx'] == res['reg_ndx'][pzdc]] -= 1 
                        normirovanyi_rabochy_den['recrut'][normirovanyi_rabochy_den['ops'] == res['ops'][pzdc]] -= 1
                        res.loc[pzdc, 'AP_NSE'] += 1
                        res.loc[pzdc, 'AP'] += 1
                        ops.loc[ops['ops'] == res['ops'][pzdc], 'nagruzka' + str(n)] += 1
                        continue
                        
                        
    #drop
    res = res.sort_values(by = 'nse_rate_drop', ascending = False).reset_index(drop = True)
    
    for i in res['region_normalny'].unique():
        for j in res['market'].dropna()[(res['region_normalny'] == i) & (res['target_drop'] != 0)]:
            if j == 'cig':
                for pzdc in res['nse_rate'][(res['region_normalny'] == i) &
                                            (res['market'].isin(['drug', 'food'])) &
                                            (res['drop'] > 0)].index:
                    if ((res['ibd_10'][res['IBD_ID'] == res['IBD_ID'][pzdc]].unique().item() >= 1) &\
                    (rot['rot'][rot['region_normalny'] == res['region_normalny'][pzdc]] >= 1) &\
                    (chanel['rot'][chanel['types'] == res['types'][pzdc]].item() >= 1) &\
                    (normirovanyi_rabochy_den['drop'][normirovanyi_rabochy_den['ops'] == res['ops'][pzdc]].item() == 1) &\
                    (index_rotation['1.5%_drop'][index_rotation['reg_ndx'] == res['reg_ndx'][pzdc]].item() >= 1)).item():
                        res.loc[pzdc, 'drop'] -= 1
                        res.loc[pzdc, 'target_drop'] += 1
                        res.loc[res['IBD_ID'] == res['IBD_ID'][pzdc], 'ibd_10'] -= 1
                        rot.loc[rot['region_normalny'] == res['region_normalny'][pzdc], 'rot'] -= 1
                        chanel.loc[chanel['types'] == res['types'][pzdc], 'rot'] -= 1
                        index_rotation['1.5%_drop'][index_rotation['reg_ndx'] == res['reg_ndx'][pzdc]] -= 1 
                        normirovanyi_rabochy_den['drop'][normirovanyi_rabochy_den['ops'] == res['ops'][pzdc]] -= 1
                        res.loc[pzdc, 'AP_NSE'] -= 1
                        res.loc[pzdc, 'AP'] -= 1
                        ops.loc[ops['ops'] == res['ops'][pzdc], 'nagruzka_drop' + str(n)] -= 1
                        continue
            elif j == 'drug':
                for pzdc in res['nse_rate'][(res['region_normalny'] == i) &
                                            (res['market'].isin(['cig', 'food'])) &
                                            (res['drop'] > 0)].index:
                    if ((res['ibd_10'][res['IBD_ID'] == res['IBD_ID'][pzdc]].unique().item() >= 1) &\
                    (rot['rot'][rot['region_normalny'] == res['region_normalny'][pzdc]] >= 1) &\
                    (chanel['rot'][chanel['types'] == res['types'][pzdc]].item() >= 1) &\
                    (normirovanyi_rabochy_den['drop'][normirovanyi_rabochy_den['ops'] == res['ops'][pzdc]].item() == 1) &\
                    (index_rotation['1.5%_drop'][index_rotation['reg_ndx'] == res['reg_ndx'][pzdc]].item() >= 1)).item():
                        res.loc[pzdc, 'drop'] -= 1
                        res.loc[pzdc, 'target_drop'] += 1
                        res.loc[res['IBD_ID'] == res['IBD_ID'][pzdc], 'ibd_10'] -= 1
                        rot.loc[rot['region_normalny'] == res['region_normalny'][pzdc], 'rot'] -= 1
                        chanel.loc[chanel['types'] == res['types'][pzdc], 'rot'] -= 1
                        index_rotation['1.5%_drop'][index_rotation['reg_ndx'] == res['reg_ndx'][pzdc]] -= 1 
                        normirovanyi_rabochy_den['drop'][normirovanyi_rabochy_den['ops'] == res['ops'][pzdc]] -= 1
                        res.loc[pzdc, 'AP_NSE'] -= 1
                        res.loc[pzdc, 'AP'] -= 1
                        ops.loc[ops['ops'] == res['ops'][pzdc], 'nagruzka_drop' + str(n)] -= 1
                        continue
            elif j == 'food':
                for pzdc in res['nse_rate'][(res['region_normalny'] == i) &
                                            (res['market'].isin(['cig', 'drug'])) &
                                            (res['drop'] > 0)].index:
                    if ((res['ibd_10'][res['IBD_ID'] == res['IBD_ID'][pzdc]].unique().item() >= 1) &\
                    (rot['rot'][rot['region_normalny'] == res['region_normalny'][pzdc]] >= 1) &\
                    (chanel['rot'][chanel['types'] == res['types'][pzdc]].item() >= 1) &\
                    (normirovanyi_rabochy_den['drop'][normirovanyi_rabochy_den['ops'] == res['ops'][pzdc]].item() == 1) &\
                    (index_rotation['1.5%_drop'][index_rotation['reg_ndx'] == res['reg_ndx'][pzdc]].item() >= 1)).item():
                        res.loc[pzdc, 'drop'] -= 1
                        res.loc[pzdc, 'target_drop'] += 1
                        res.loc[res['IBD_ID'] == res['IBD_ID'][pzdc], 'ibd_10'] -= 1
                        rot.loc[rot['region_normalny'] == res['region_normalny'][pzdc], 'rot'] -= 1
                        chanel.loc[chanel['types'] == res['types'][pzdc], 'rot'] -= 1
                        index_rotation['1.5%_drop'][index_rotation['reg_ndx'] == res['reg_ndx'][pzdc]] -= 1 
                        normirovanyi_rabochy_den['drop'][normirovanyi_rabochy_den['ops'] == res['ops'][pzdc]] -= 1
                        res.loc[pzdc, 'AP_NSE'] -= 1
                        res.loc[pzdc, 'AP'] -= 1
                        ops.loc[ops['ops'] == res['ops'][pzdc], 'nagruzka_drop' + str(n)] -= 1
                        continue
            elif j == 'food+cig':
                for pzdc in res['nse_rate'][(res['region_normalny'] == i) &
                                            (res['market'].isin(['drug'])) &
                                            (res['drop'] > 0)].index:
                    if ((res['ibd_10'][res['IBD_ID'] == res['IBD_ID'][pzdc]].unique().item() >= 1) &\
                    (rot['rot'][rot['region_normalny'] == res['region_normalny'][pzdc]] >= 1) &\
                    (chanel['rot'][chanel['types'] == res['types'][pzdc]].item() >= 1) &\
                    (normirovanyi_rabochy_den['drop'][normirovanyi_rabochy_den['ops'] == res['ops'][pzdc]].item() == 1) &\
                    (index_rotation['1.5%_drop'][index_rotation['reg_ndx'] == res['reg_ndx'][pzdc]].item() >= 1)).item():
                        res.loc[pzdc, 'drop'] -= 1
                        res.loc[pzdc, 'target_drop'] += 1
                        res.loc[res['IBD_ID'] == res['IBD_ID'][pzdc], 'ibd_10'] -= 1
                        rot.loc[rot['region_normalny'] == res['region_normalny'][pzdc], 'rot'] -= 1
                        chanel.loc[chanel['types'] == res['types'][pzdc], 'rot'] -= 1
                        index_rotation['1.5%_drop'][index_rotation['reg_ndx'] == res['reg_ndx'][pzdc]] -= 1 
                        normirovanyi_rabochy_den['drop'][normirovanyi_rabochy_den['ops'] == res['ops'][pzdc]] -= 1
                        res.loc[pzdc, 'AP_NSE'] -= 1
                        res.loc[pzdc, 'AP'] -= 1
                        ops.loc[ops['ops'] == res['ops'][pzdc], 'nagruzka_drop' + str(n)] -= 1
                        continue            
                        
                        
                        
    return res, ops   

In [64]:
res1, ops = trend(df, ops, 1)
res2, ops = trend(res1, ops, 2)
res3, ops = trend(res2, ops, 3)
res4, ops = trend(res3, ops, 4)
res5, ops = trend(res4, ops, 5)
res6, ops = trend(res5, ops, 6)
res7, ops = trend(res6, ops, 7)
res8, ops = trend(res7, ops, 8)
res9, ops = trend(res8, ops, 9)
res10, ops = trend(res9, ops, 10)
res11, ops = trend(res10, ops, 11)
res12, ops = trend(res11, ops, 12)
res13, ops = trend(res12, ops, 13)
res14, ops = trend(res13, ops, 14)
res15, ops  = trend(res14, ops, 15)
res16, ops  = trend(res15, ops, 16)
res17, ops  = trend(res16, ops, 17)
res18, ops  = trend(res17, ops, 18)

C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:53: FutureWarning: `item` has been deprecated and will be removed in a future version
C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:54: FutureWarning: `item` has been deprecated and will be removed in a future version
C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:99: RuntimeWarning: invalid value encountered in longlong_scalars
C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:87: RuntimeWarning: invalid value encountered in longlong_scalars
C:

C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:313: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:306: FutureWarning: `item` has been deprecated and will be removed in a future version
C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:307: FutureWarning: `item` has been deprecated and will be removed in a future version
C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:266: FutureWarning: `item` has been deprecated and will be removed in a future version
C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:267: FutureWarning: `item` has been deprecat

C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:400: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:353: FutureWarning: `item` has been deprecated and will be removed in a future version
C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:354: FutureWarning: `item` has been deprecated and will be removed in a future version
C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:360: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ve

C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:266: FutureWarning: `item` has been deprecated and will be removed in a future version
C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:267: FutureWarning: `item` has been deprecated and will be removed in a future version
C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:306: FutureWarning: `item` has been deprecated and will be removed in a future version
C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:307: FutureWarning: `item` has been deprecated and will be removed in a future version
C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:246: FutureWarning: `item` has been deprecated and will be removed in a future version
C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:247: FutureWa

C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:334: FutureWarning: `item` has been deprecated and will be removed in a future version
C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:340: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:333: FutureWarning: `item` has been deprecated and will be removed in a future version
C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:334: FutureWarning: `item` has been deprecated and will be removed in a future version
C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:373: FutureWarning: `item` has been deprecat

...

C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:266: FutureWarning: `item` has been deprecated and will be removed in a future version
C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:267: FutureWarning: `item` has been deprecated and will be removed in a future version
C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:306: FutureWarning: `item` has been deprecated and will be removed in a future version
C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:307: FutureWarning: `item` has been deprecated and will be removed in a future version
C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:286: FutureWarning: `item` has been deprecated and will be removed in a future version
C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:287: FutureWa

vse


C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:266: FutureWarning: `item` has been deprecated and will be removed in a future version
C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:267: FutureWarning: `item` has been deprecated and will be removed in a future version
C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:313: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:306: FutureWarning: `item` has been deprecated and will be removed in a future version
C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:307: FutureWarning: `item` has been deprecat

vse


C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:286: FutureWarning: `item` has been deprecated and will be removed in a future version
C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:287: FutureWarning: `item` has been deprecated and will be removed in a future version
C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:266: FutureWarning: `item` has been deprecated and will be removed in a future version
C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:267: FutureWarning: `item` has been deprecated and will be removed in a future version


In [65]:
res19, ops = trend(res18, ops, 19)
res20, ops = trend(res19, ops, 20)
res21, ops = trend(res20, ops, 21)
res22, ops = trend(res21, ops, 22)
res23, ops = trend(res22, ops, 23)
res24, ops = trend(res23, ops, 24)

C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:53: FutureWarning: `item` has been deprecated and will be removed in a future version
C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:54: FutureWarning: `item` has been deprecated and will be removed in a future version
C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:99: RuntimeWarning: invalid value encountered in longlong_scalars
C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:87: RuntimeWarning: invalid value encountered in longlong_scalars
C:

vse


C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:306: FutureWarning: `item` has been deprecated and will be removed in a future version
C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:307: FutureWarning: `item` has been deprecated and will be removed in a future version
C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:313: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:306: FutureWarning: `item` has been deprecated and will be removed in a future version
C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:307: FutureWarning: `item` has been deprecat

vse
vse
vse
vse
vse
vse
vse
vse
vse


In [66]:
res1 = res1.rename(columns = {'target': 'target1', 'target_drop': 'target_drop1'})
res2 = res2.rename(columns = {'target': 'target2', 'target_drop': 'target_drop2'})
res3 = res3.rename(columns = {'target': 'target3', 'target_drop': 'target_drop3'})
res4 = res4.rename(columns = {'target': 'target4', 'target_drop': 'target_drop4'})
res5 = res5.rename(columns = {'target': 'target5', 'target_drop': 'target_drop5'})
res6 = res6.rename(columns = {'target': 'target6', 'target_drop': 'target_drop6'})
res7 = res7.rename(columns = {'target': 'target7', 'target_drop': 'target_drop7'})
res8 = res8.rename(columns = {'target': 'target8', 'target_drop': 'target_drop8'})
res9 = res9.rename(columns = {'target': 'target9', 'target_drop': 'target_drop9'})
res10 = res10.rename(columns = {'target': 'target10', 'target_drop': 'target_drop10'})
res11 = res11.rename(columns = {'target': 'target11', 'target_drop': 'target_drop11'})
res12 = res12.rename(columns = {'target': 'target12', 'target_drop': 'target_drop12'})
res13 = res13.rename(columns = {'target': 'target13', 'target_drop': 'target_drop13'})
res14 = res14.rename(columns = {'target': 'target14', 'target_drop': 'target_drop14'})
res15 = res15.rename(columns = {'target': 'target15', 'target_drop': 'target_drop15'})
res16 = res16.rename(columns = {'target': 'target16', 'target_drop': 'target_drop16'})
res17 = res17.rename(columns = {'target': 'target17', 'target_drop': 'target_drop17'})
res18 = res18.rename(columns = {'target': 'target18', 'target_drop': 'target_drop18'})
res19 = res19.rename(columns = {'target': 'target19', 'target_drop': 'target_drop19'})
res20 = res20.rename(columns = {'target': 'target20', 'target_drop': 'target_drop20'})
res21 = res21.rename(columns = {'target': 'target21', 'target_drop': 'target_drop21'})
res22 = res22.rename(columns = {'target': 'target22', 'target_drop': 'target_drop22'})
res23 = res23.rename(columns = {'target': 'target23', 'target_drop': 'target_drop23'})
res24 = res24.rename(columns = {'target': 'target24', 'target_drop': 'target_drop24'})

In [67]:
df = pd.merge(df, res1[['index', 'target1', 'target_drop1']])
df = pd.merge(df, res2[['index', 'target2', 'target_drop2']])
df = pd.merge(df, res3[['index', 'target3', 'target_drop3']])
df = pd.merge(df, res4[['index', 'target4', 'target_drop4']])
df = pd.merge(df, res5[['index', 'target5', 'target_drop5']])
df = pd.merge(df, res6[['index', 'target6', 'target_drop6']])
df = pd.merge(df, res7[['index', 'target7', 'target_drop7']])
df = pd.merge(df, res8[['index', 'target8', 'target_drop8']])
df = pd.merge(df, res9[['index', 'target9', 'target_drop9']])
df = pd.merge(df, res10[['index', 'target10', 'target_drop10']])
df = pd.merge(df, res11[['index', 'target11', 'target_drop11']])
df = pd.merge(df, res12[['index', 'target12', 'target_drop12']])
df = pd.merge(df, res13[['index', 'target13', 'target_drop13']])
df = pd.merge(df, res14[['index', 'target14', 'target_drop14']])
df = pd.merge(df, res15[['index', 'target15', 'target_drop15']])
df = pd.merge(df, res16[['index', 'target16', 'target_drop16']])
df = pd.merge(df, res17[['index', 'target17', 'target_drop17']])
df = pd.merge(df, res18[['index', 'target18', 'target_drop18']])
df = pd.merge(df, res19[['index', 'target19', 'target_drop19']])
df = pd.merge(df, res20[['index', 'target20', 'target_drop20']])
df = pd.merge(df, res21[['index', 'target21', 'target_drop21']])
df = pd.merge(df, res22[['index', 'target22', 'target_drop22']])
df = pd.merge(df, res23[['index', 'target23', 'target_drop23']])
df = pd.merge(df, res24[['index', 'target24', 'target_drop24']])

In [68]:
with pd.ExcelWriter('output.xlsx') as writer:
    df.to_excel(writer, sheet_name='ini')
    res1.to_excel(writer, sheet_name='1')
    res2.to_excel(writer, sheet_name='2')
    res3.to_excel(writer, sheet_name='3')
    res4.to_excel(writer, sheet_name='4')
    res5.to_excel(writer, sheet_name='5')
    res6.to_excel(writer, sheet_name='6')
    res7.to_excel(writer, sheet_name='7')
    res8.to_excel(writer, sheet_name='8')
    res9.to_excel(writer, sheet_name='9')
    res10.to_excel(writer, sheet_name='10')
    res11.to_excel(writer, sheet_name='11')
    res12.to_excel(writer, sheet_name='12')
    res13.to_excel(writer, sheet_name='13')
    res14.to_excel(writer, sheet_name='14')
    res15.to_excel(writer, sheet_name='15')
    res16.to_excel(writer, sheet_name='16')
    res17.to_excel(writer, sheet_name='17')
    res18.to_excel(writer, sheet_name='18')
    res19.to_excel(writer, sheet_name='19')
    res20.to_excel(writer, sheet_name='20')
    res21.to_excel(writer, sheet_name='21')
    res22.to_excel(writer, sheet_name='22')
    res23.to_excel(writer, sheet_name='23')
    res24.to_excel(writer, sheet_name='24')
    ops.to_excel(writer, sheet_name = 'ops')